<a href="https://colab.research.google.com/github/PalomiBollapalli/Hackathon/blob/main/hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import subprocess
import os

def simple_fuzzer(target_program, iterations=10000, max_length=100):
    """A simple fuzzer that generates random byte strings."""
    for i in range(iterations):
        length = random.randint(0, max_length)
        input_data = bytes([random.randint(0, 255) for _ in range(length)])

        try:
            result = subprocess.run([target_program], input=input_data, capture_output=True, timeout=1)  # timeout for safety
            if result.returncode < 0 : # negative return codes indicate signals (crashes)
              print(f"Crash detected! Input: {input_data}")
              with open("crash.bin", "wb") as f:
                f.write(input_data)
              return
            elif result.returncode != 0:
              print(f"Non-zero return code: {result.returncode}, Input: {input_data}") # report non-crash errors
        except subprocess.TimeoutExpired:
            print(f"Timeout! Input: {input_data}")
            with open("timeout.bin", "wb") as f:
              f.write(input_data)
            return
        except FileNotFoundError:
          print(f"Target program not found: {target_program}")
          return
        except Exception as e:
            print(f"An error occurred: {e}")
            return
    print("Fuzzing completed without crashes.")

def mutation_fuzzer(target_program, iterations=10000, initial_input=b"initial_input"):
    """A fuzzer that mutates an initial input."""
    input_data = bytearray(initial_input)

    for i in range(iterations):
        mutated_data = bytearray(input_data)  # Create a copy for mutation
        num_mutations = random.randint(1, 5)

        for _ in range(num_mutations):
            mutation_type = random.randint(0, 2)
            mutation_pos = random.randint(0, len(mutated_data) - 1) if mutated_data else 0 # handle empty input

            if mutation_type == 0:  # Bit flip
                mutated_data[mutation_pos] ^= 1 << random.randint(0, 7)
            elif mutation_type == 1:  # Byte overwrite
                mutated_data[mutation_pos] = random.randint(0, 255)
            elif mutation_type == 2 and len(mutated_data) < 200: #byte insertion, with size limit.
                mutated_data.insert(mutation_pos, random.randint(0, 255))
        try:
            result = subprocess.run([target_program], input=bytes(mutated_data), capture_output=True, timeout=1)
            if result.returncode < 0:
                print(f"Crash detected! Input: {mutated_data}")
                with open("crash.bin", "wb") as f:
                    f.write(mutated_data)
                return
            elif result.returncode != 0:
              print(f"Non-zero return code: {result.returncode}, Input: {mutated_data}")
        except subprocess.TimeoutExpired:
            print(f"Timeout! Input: {mutated_data}")
            with open("timeout.bin", "wb") as f:
                f.write(mutated_data)
            return
        except FileNotFoundError:
          print(f"Target program not found: {target_program}")
          return
        except Exception as e:
            print(f"An error occurred: {e}")
            return

        input_data = mutated_data  # Update input for the next iteration
    print("Fuzzing completed without crashes.")

if __name__ == "__main__":
    target = "./vulnerable_program" # Replace with your target program.
    if os.name == 'nt': # windows testing
        target = "vulnerable_program.exe"

    #Create a simple, vulnerable program for testing.
    with open("vulnerable_program.c", "w") as f:
        f.write("""
        #include <stdio.h>
        #include <string.h>
        #include <stdlib.h>
        int main(int argc, char *argv[]) {
            char buffer[10];
            if (argc > 1) {
                strcpy(buffer, argv[1]);
            }
            return 0;
        }
        """)

    subprocess.run(["gcc", "vulnerable_program.c", "-o", target])

    print("Running simple fuzzer...")
    simple_fuzzer(target)

    print("\nRunning mutation fuzzer...")
    mutation_fuzzer(target)

    os.remove("vulnerable_program.c")
    os.remove(target) #cleanup

Running simple fuzzer...
Fuzzing completed without crashes.

Running mutation fuzzer...
Fuzzing completed without crashes.
